In [1]:
## Load relevant R packages.

inLibraries = list('repr','spdep','raster','rgdal','rgeos')
for (rpack in inLibraries) {
  if (is.element(rpack,installed.packages()[,1])){
      #Load the library into R
      suppressMessages(library(rpack,character.only = TRUE))
    }
    else {
        print(paste("Warning:  ",rpack," is not an installed package"))
    }
}

##  Resize plot area.

options(repr.plot.width=6, repr.plot.height=6)

library(geojsonio)


Registered S3 method overwritten by 'geojsonsf':
  method        from   
  print.geojson geojson


Attaching package: ‘geojsonio’


The following object is masked from ‘package:base’:

    pretty




In [2]:
meteorites <- read.csv("data/results/meteorites.csv",stringsAsFactors=FALSE)
meteorites <- subset(meteorites, select = -c(X) )

population <- read.csv("data/results/meteorites_population.csv",stringsAsFactors=FALSE)
population <- subset(population, select = -c(X) )

landcover <- read.csv("data/results/meteorites_landcover.csv",stringsAsFactors=FALSE)
landcover <- subset(landcover, select = -c(X) )

head(meteorites)
head(population)
head(landcover)

,name,id,nametype,recclass,mass_in_grams,fell_or_found,year,latitude,longitude,GeoLocation,group_name,Chondrite.Achondrite,type,L3,L4
,<chr>,<int>,<chr>,<chr>,<dbl>,<chr>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Aachen,1,Valid,L5,21,Fell,1880,50.77500,6.08333,"(50.775, 6.08333)",L,Chondrite,Stony,Ordinary,L
2,Aarhus,2,Valid,H6,720,Fell,1951,56.18333,10.23333,"(56.18333, 10.23333)",H,Chondrite,Stony,Ordinary,H
3,Abajo,4,Valid,H5,331,Found,1982,26.80000,-105.41667,"(26.8, -105.41667)",H,Chondrite,Stony,Ordinary,H
4,Abbott,5,Valid,H3-6,21100,Found,1951,36.30000,-104.28333,"(36.3, -104.28333)",H,Chondrite,Stony,Ordinary,H
5,Abee,6,Valid,EH4,107000,Fell,1952,54.21667,-113.00000,"(54.21667, -113.0)",EH,Chondrite,Stony,Enstatite,EH-EL
6,Abernathy,7,Valid,L6,2914,Found,1941,33.85000,-101.80000,"(33.85, -101.8)",L,Chondrite,Stony,Ordinary,L


,x,y,name,id,year,closest_city,pop,pop_year
,<dbl>,<dbl>,<chr>,<int>,<int>,<chr>,<int>,<int>
1,6.08333,50.77500,Aachen,1,1880,Aachen,48688,1851
2,10.23333,56.18333,Aarhus,2,1951,Aarhus,51000,1900
3,-105.41667,26.80000,Abajo,4,1982,NA,NA,NA
4,-104.28333,36.30000,Abbott,5,1951,NA,NA,NA
5,-113.00000,54.21667,Abee,6,1952,NA,NA,NA
6,-101.80000,33.85000,Abernathy,7,1941,NA,NA,NA


,id,name,lc_sample
,<int>,<chr>,<int>
1,1,Aachen,11
2,2,Aarhus,1
3,4,Abajo,2
4,5,Abbott,13
5,6,Abee,1
6,7,Abernathy,1


In [3]:
meteorites_lc <- merge(meteorites,landcover,by="id")

head(meteorites_lc)

,id,name.x,nametype,recclass,mass_in_grams,fell_or_found,year,latitude,longitude,GeoLocation,group_name,Chondrite.Achondrite,type,L3,L4,name.y,lc_sample
,<int>,<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
1,1,Aachen,Valid,L5,21,Fell,1880,50.77500,6.08333,"(50.775, 6.08333)",L,Chondrite,Stony,Ordinary,L,Aachen,11
2,2,Aarhus,Valid,H6,720,Fell,1951,56.18333,10.23333,"(56.18333, 10.23333)",H,Chondrite,Stony,Ordinary,H,Aarhus,1
3,4,Abajo,Valid,H5,331,Found,1982,26.80000,-105.41667,"(26.8, -105.41667)",H,Chondrite,Stony,Ordinary,H,Abajo,2
4,5,Abbott,Valid,H3-6,21100,Found,1951,36.30000,-104.28333,"(36.3, -104.28333)",H,Chondrite,Stony,Ordinary,H,Abbott,13
5,6,Abee,Valid,EH4,107000,Fell,1952,54.21667,-113.00000,"(54.21667, -113.0)",EH,Chondrite,Stony,Enstatite,EH-EL,Abee,1
6,7,Abernathy,Valid,L6,2914,Found,1941,33.85000,-101.80000,"(33.85, -101.8)",L,Chondrite,Stony,Ordinary,L,Abernathy,1


In [4]:
megatable <- merge(meteorites_lc,population,by="id")

keep <- c('id',
          'nametype',
          'recclass',
          'mass_in_grams',
          'fell_or_found',
          'latitude',
          'longitude',
          'GeoLocation',
          'group_name',
          'Chondrite.Achondrite',
          'type',
          'L3',
          'L4',
          'lc_sample',
          'closest_city',
          'pop', 
          'pop_year')

megatable2 <- subset(megatable, select = c(keep))

megatable2

,id,nametype,recclass,mass_in_grams,fell_or_found,latitude,longitude,GeoLocation,group_name,Chondrite.Achondrite,type,L3,L4,lc_sample,closest_city,pop,pop_year
,<int>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>
1,1,Valid,L5,21.0,Fell,50.77500,6.08333,"(50.775, 6.08333)",L,Chondrite,Stony,Ordinary,L,11,Aachen,48688,1851
2,2,Valid,H6,720.0,Fell,56.18333,10.23333,"(56.18333, 10.23333)",H,Chondrite,Stony,Ordinary,H,1,Aarhus,51000,1900
3,4,Valid,H5,331.0,Found,26.80000,-105.41667,"(26.8, -105.41667)",H,Chondrite,Stony,Ordinary,H,2,NA,NA,NA
4,5,Valid,H3-6,21100.0,Found,36.30000,-104.28333,"(36.3, -104.28333)",H,Chondrite,Stony,Ordinary,H,13,NA,NA,NA
5,6,Valid,EH4,107000.0,Fell,54.21667,-113.00000,"(54.21667, -113.0)",EH,Chondrite,Stony,Enstatite,EH-EL,1,NA,NA,NA
6,7,Valid,L6,2914.0,Found,33.85000,-101.80000,"(33.85, -101.8)",L,Chondrite,Stony,Ordinary,L,1,NA,NA,NA
7,8,Valid,H,1.2,Found,60.43333,22.30000,"(60.43333, 22.3)",H,Chondrite,Stony,Ordinary,H,1,NA,NA,NA
8,9,Valid,H4,4500.0,Found,27.23944,29.83583,"(27.23944, 29.83583)",H,Chondrite,Stony,Ordinary,H,11,NA,NA,NA
9,10,Valid,Acapulcoite,1914.0,Fell,16.88333,-99.90000,"(16.88333, -99.9)",Acapulcoite,Achondrite,Stony,Primitive,-,0,NA,NA,NA


In [5]:
# Creating a score to weight meteorite finds.

# I want there to be a weighting where meteorites found in rare places are "worth more" than those found 
# in obvious places.

# First looking at the population:

urban <- subset(megatable2, (!is.na(megatable$pop)))
summary(urban)
urban

       id          nametype           recclass         mass_in_grams     
 Min.   :    1   Length:239         Length:239         Min.   :       0  
 1st Qu.:10851   Class :character   Class :character   1st Qu.:     367  
 Median :16627   Mode  :character   Mode  :character   Median :    1500  
 Mean   :17644                                         Mean   :  108967  
 3rd Qu.:23758                                         3rd Qu.:    8750  
 Max.   :55584                                         Max.   :15500000  
 fell_or_found         latitude        longitude        GeoLocation       
 Length:239         Min.   :-34.55   Min.   :-157.783   Length:239        
 Class :character   1st Qu.: 28.50   1st Qu.:  -6.533   Class :character  
 Mode  :character   Median : 38.18   Median :  13.167   Mode  :character  
                    Mean   : 35.21   Mean   :  17.778                     
                    3rd Qu.: 46.30   3rd Qu.:  76.282                     
                    Max.   : 58.

,id,nametype,recclass,mass_in_grams,fell_or_found,latitude,longitude,GeoLocation,group_name,Chondrite.Achondrite,type,L3,L4,lc_sample,closest_city,pop,pop_year
,<int>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>
1,1,Valid,L5,21.0,Fell,50.77500,6.08333,"(50.775, 6.08333)",L,Chondrite,Stony,Ordinary,L,11,Aachen,48688,1851
2,2,Valid,H6,720.0,Fell,56.18333,10.23333,"(56.18333, 10.23333)",H,Chondrite,Stony,Ordinary,H,1,Aarhus,51000,1900
415,425,Valid,Unknown,367.0,Fell,50.66667,2.33333,"(50.66667, 2.33333)",Unknown,-,-,-,-,1,Saint-Omer,20000,1300
436,446,Valid,CR2-an,160.0,Fell,24.41667,39.51667,"(24.41667, 39.51667)",CR,Chondrite,Stony,Carbonaceous,CR,2,Medina,27500,1900
451,462,Valid,L6,3200.0,Fell,36.23333,37.13333,"(36.23333, 37.13333)",L,Chondrite,Stony,Ordinary,L,2,Aleppo,99383,1856
452,463,Valid,H5,908.0,Fell,44.88333,8.75000,"(44.88333, 8.75)",H,Chondrite,Stony,Ordinary,H,1,Alessandria,22994,1848
455,466,Valid,L6,228000.0,Fell,45.26667,10.15000,"(45.26667, 10.15)",L,Chondrite,Stony,Ordinary,L,1,Cremona,28328,1851
485,2301,Valid,L6,367.0,Fell,47.46667,-0.55000,"(47.46667, -0.55)",L,Chondrite,Stony,Ordinary,L,1,Angers,25442,1809
497,2315,Valid,L6,6.1,Found,39.75000,-105.20000,"(39.75, -105.2)",L,Chondrite,Stony,Ordinary,L,1,Denver,272000,1925


In [ ]:
# 239 of the hits were near an urban area.  Honestly, I don't believe 
# that's going to affect my results very much.  

In [6]:
#next, landcover.  These are the land values that meteorites have been found in:

#   11  Temperate deciduous forest
#    2  Pasture/land used for grazing
#    1  Cultivated land
#   14  Hot desert
#   13  Grassland/Steppe
#   10  Temperate mixed forest
#    0  Water Bodies
#   15  Scrubland
#   12  Warm mixed forest
#   16  Savanna
#    8  Boreal forest
#    9  Cool conifer forest
#   17  Tropical woodland
#   18  Tropical forest
#    7  Wooded tundra
#    6  Tundra

# and here are the percentages for them:

library(dplyr)

counts <- megatable2 %>% group_by(lc_sample) %>% summarise(n=n())

total <- nrow(megatable2)

for (i in 1:16){
    value <- counts[i,2]
    value_percent <- (value/total)*100
    counts[i,3] <- value_percent
}

counts <- counts[,-2]
counts


Attaching package: ‘dplyr’


The following objects are masked from ‘package:rgeos’:

    intersect, setdiff, union


The following objects are masked from ‘package:raster’:

    intersect, select, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




lc_sample,n
<int>,<dbl>
0,0.9295746
1,7.6790947
2,9.8918864
6,0.1212489
7,0.0707285
8,0.4344751
9,0.2121855
10,1.1720723
11,69.2533091


In [7]:
#  0		  92		 0.9295746	Water Bodies
#  1		 760		 7.6790947	Cultivated land
#  2		 979		 9.8918864	Pasture/land used for grazing
#  6		  12		 0.1212489	Tundra
#  7		   7		 0.0707285	Wooded tundra
#  8		  43		 0.4344751	Boreal forest
#  9		  21		 0.2121855	Cool conifer forest
# 10		 116		 1.1720723	Temperate mixed forest
# 11		6854		69.2533091	Temperate deciduous forest
# 12		  71		 0.7173891	Warm mixed forest
# 13		 331		 3.3444478	Grassland/Steppe
# 14		 442		 4.4659998	Hot desert
# 15		  83		 0.8386380	Scrubland
# 16		  57		 0.5759321	Savanna
# 17		  19		 0.1919774	Tropical woodland
# 18		  10		 0.1010407	Tropical forest

In [8]:
# How does this compare to the actual land coverage?

lc1990name <- "data/lc/historic_landcover_hd_1990.asc"
lc1990 <- raster(lc1990name,values=TRUE)

counts2 <- freq(lc1990)

#lc1990

total2 <- 259200

for (i in 1:18){
    value2 <- counts2[i,2]
    value2_percent <- (value2/total2)*100
    counts2[i,2] <- value2_percent
}

counts2

value,count
0,66.3206019
1,2.2827932
2,4.5821759
5,1.1292438
6,1.8429784
7,0.7133488
8,4.0671296
9,0.6041667
10,0.5412809
11,0.3364198


In [9]:
# no category 5 or 19....

counts3 <- counts2[-4,]
counts3 <- counts3[-17,]

counts3 <- as.data.frame(counts3)

over_under <- merge(counts, counts3, by.x= "lc_sample", by.y="value")
over_under

lc_sample,n,count
<int>,<dbl>,<dbl>
0,0.9295746,66.3206019
1,7.6790947,2.2827932
2,9.8918864,4.5821759
6,0.1212489,1.8429784
7,0.0707285,0.7133488
8,0.4344751,4.0671296
9,0.2121855,0.6041667
10,1.1720723,0.5412809
11,69.2533091,0.3364198


In [10]:
over_under$over_under <- over_under$count/over_under$n

over_under

lc_sample,n,count,over_under
<int>,<dbl>,<dbl>,<dbl>
0,0.9295746,66.3206019,71.345108318
1,7.6790947,2.2827932,0.297273742
2,9.8918864,4.5821759,0.463225691
6,0.1212489,1.8429784,15.199964313
7,0.0707285,0.7133488,10.085732474
8,0.4344751,4.0671296,9.361019057
9,0.2121855,0.6041667,2.847351190
10,1.1720723,0.5412809,0.461815234
11,69.2533091,0.3364198,0.004857815


In [11]:
# A value > 1 means that the samples underrepresent the earth's land cover, 
# while a value < 1 means the samples overrepresent.  These weights will be important to add for predictions.

head(megatable)
head(megatable2)

,id,name.x,nametype,recclass,mass_in_grams,fell_or_found,year.x,latitude,longitude,GeoLocation,⋯,L4,name.y,lc_sample,x,y,name,year.y,closest_city,pop,pop_year
,<int>,<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<dbl>,<dbl>,<chr>,⋯,<chr>,<chr>,<int>,<dbl>,<dbl>,<chr>,<int>,<chr>,<int>,<int>
1,1,Aachen,Valid,L5,21,Fell,1880,50.77500,6.08333,"(50.775, 6.08333)",⋯,L,Aachen,11,6.08333,50.77500,Aachen,1880,Aachen,48688,1851
2,2,Aarhus,Valid,H6,720,Fell,1951,56.18333,10.23333,"(56.18333, 10.23333)",⋯,H,Aarhus,1,10.23333,56.18333,Aarhus,1951,Aarhus,51000,1900
3,4,Abajo,Valid,H5,331,Found,1982,26.80000,-105.41667,"(26.8, -105.41667)",⋯,H,Abajo,2,-105.41667,26.80000,Abajo,1982,NA,NA,NA
4,5,Abbott,Valid,H3-6,21100,Found,1951,36.30000,-104.28333,"(36.3, -104.28333)",⋯,H,Abbott,13,-104.28333,36.30000,Abbott,1951,NA,NA,NA
5,6,Abee,Valid,EH4,107000,Fell,1952,54.21667,-113.00000,"(54.21667, -113.0)",⋯,EH-EL,Abee,1,-113.00000,54.21667,Abee,1952,NA,NA,NA
6,7,Abernathy,Valid,L6,2914,Found,1941,33.85000,-101.80000,"(33.85, -101.8)",⋯,L,Abernathy,1,-101.80000,33.85000,Abernathy,1941,NA,NA,NA


,id,nametype,recclass,mass_in_grams,fell_or_found,latitude,longitude,GeoLocation,group_name,Chondrite.Achondrite,type,L3,L4,lc_sample,closest_city,pop,pop_year
,<int>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>
1,1,Valid,L5,21,Fell,50.77500,6.08333,"(50.775, 6.08333)",L,Chondrite,Stony,Ordinary,L,11,Aachen,48688,1851
2,2,Valid,H6,720,Fell,56.18333,10.23333,"(56.18333, 10.23333)",H,Chondrite,Stony,Ordinary,H,1,Aarhus,51000,1900
3,4,Valid,H5,331,Found,26.80000,-105.41667,"(26.8, -105.41667)",H,Chondrite,Stony,Ordinary,H,2,NA,NA,NA
4,5,Valid,H3-6,21100,Found,36.30000,-104.28333,"(36.3, -104.28333)",H,Chondrite,Stony,Ordinary,H,13,NA,NA,NA
5,6,Valid,EH4,107000,Fell,54.21667,-113.00000,"(54.21667, -113.0)",EH,Chondrite,Stony,Enstatite,EH-EL,1,NA,NA,NA
6,7,Valid,L6,2914,Found,33.85000,-101.80000,"(33.85, -101.8)",L,Chondrite,Stony,Ordinary,L,1,NA,NA,NA


In [17]:
over_under2 <- subset(over_under, select= -c(2,3))
over_under2

,lc_sample,over_under
,<int>,<dbl>
1,0,71.345108318
2,1,0.297273742
3,2,0.463225691
4,6,15.199964313
5,7,10.085732474
6,8,9.361019057
7,9,2.847351190
8,10,0.461815234
9,11,0.004857815


In [36]:
megatable3 <- merge (megatable2, over_under2, by="lc_sample")

megatable3 <- megatable3[order(megatable3$id),]

megatable4 <- megatable3[, c(2:14,1,18)]

head(megatable4)



,id,nametype,recclass,mass_in_grams,fell_or_found,latitude,longitude,GeoLocation,group_name,Chondrite.Achondrite,type,L3,L4,lc_sample,over_under
,<int>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>
2031,1,Valid,L5,21,Fell,50.77500,6.08333,"(50.775, 6.08333)",L,Chondrite,Stony,Ordinary,L,11,0.004857815
285,2,Valid,H6,720,Fell,56.18333,10.23333,"(56.18333, 10.23333)",H,Chondrite,Stony,Ordinary,H,1,0.297273742
1759,4,Valid,H5,331,Found,26.80000,-105.41667,"(26.8, -105.41667)",H,Chondrite,Stony,Ordinary,H,2,0.463225691
9266,5,Valid,H3-6,21100,Found,36.30000,-104.28333,"(36.3, -104.28333)",H,Chondrite,Stony,Ordinary,H,13,0.428663282
493,6,Valid,EH4,107000,Fell,54.21667,-113.00000,"(54.21667, -113.0)",EH,Chondrite,Stony,Enstatite,EH-EL,1,0.297273742
411,7,Valid,L6,2914,Found,33.85000,-101.80000,"(33.85, -101.8)",L,Chondrite,Stony,Ordinary,L,1,0.297273742


In [37]:
write.csv(megatable4,'data/results/meteorites_weighted.csv')

